Christian Young<br>
ML Assignment<br>
26 March 2025<br>
# Terms/Context
- **NLQ** - Natural language query. This refers to the query that the user would provide e.g. "How many players are there"
- **T2SQL** - Text-to-sql, when I say T2SQL I am referring to the model that I am using to convert a NL input to SQL
- **Table of contents** - For convenience, each section has a link at the top to return to the table of contents

# Summary
### Major points of progress:
**1. Dynamic calculation of relevant tables/columns**<br>
_Issue_: The T2SQL model needs the relevant tables/models provided as context so that the model can generate correct SQL. This means if a user makes an NLQ, they would also have to specify which tables/columns they want to query (major inconvenience). With too many tables/columns provided as context, the model would not be able to correctly identify which tables/columns are the most relevant to the NLQ because of all the noise.<br><br>
_Solution_: I implemented a Retrieval-Augmented Generation (RAG) architecture that dynamically calculates which tables/columns are relevant based on the NLQ. Now the user only has to provide their NQL. Granted, there is still fine-tuning that needs to be done, because there are queries that the model still struggles with.<br><br>
**2. Fine-tuning model**<br>
_Issue_: The T2SQL model struggled with some queries because the model was not trained specifically on our dataset<br><br>
_Solution_: Leveraged parameter hypertuning to help fine-tune this model

# Document overview

[Section 1 Building rag architecture](#Section-1-rag-implementation)  
&nbsp;&nbsp;- [Convert db schema and nlq to vector embeddings](#Convert-db-schema-and-nlq-to-vector-embeddings)  
&nbsp;&nbsp;- [Perform similarity search](#Perform-similarity-search)  
&nbsp;&nbsp;- [Convert search results to input prompt](#Convert-search-results-to-input-prompt)  

[Section 2 Transfer learning](#Section-2-transfer-learning)  
&nbsp;&nbsp;- [Test train split and data preprocessing](#Test-train-split-and-data-preprocessing)  
&nbsp;&nbsp;- [Hyperparameter tuning and training](#Hyperparameter-tuning-and-training)

[Section 3 Putting it all together](#Section-3-putting-it-all-together)

[Section 4 Resources](#Section-4-resources)  

In [1]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer, InputExample, losses
import sqlite3
import re
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer,  AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Section 1 rag implementation
[Top](#Document-overview)

### Convert db schema and nlq to vector embeddings
[Top](#Document-overview)

**Embedding DB Schema**<br>
In the excel file I have each data that includes column name, column description, and table name. This is all put in one string per column and that string is turned into a vector embedding. Each vector embedding has the same length<br><br>
**Embedding NL query**<br>
Similarly, each NLQ is being converted to vector embeddings of length 384. This will make similarity comparison easier.
<br><br>

In a production environment these would be stored in a vector database, but I keep them as variables in my script so that I can more quickly develop and prototype solutions.

In [2]:
# list of columns with descriptions
db_columns = pd.read_excel("../data/data_sheets.xlsx", sheet_name="schema_metadata")["full_column_metadata"].tolist()

# convert db metadata to vector embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
db_schema_embeddings = embedding_model.encode(db_columns)

# convert NL queries to vector embedding
nl_queries = [
    "Who is the tallest player?",
    "Who is the heaviest player?",
    "What is the name of the tallest player?",
    "How many players are there?",
    "How many countries are there",
    "How many leagues are there?",
    "What is the average player height?",
    "What is the average player weight?",
]
nl_queries_embeddings = embedding_model.encode(nl_queries)

print(f"All 12 DB columns:")
for col in db_columns:
    print(col)
print(f"\nDB schema shape: {db_schema_embeddings.shape}")
print(f"NL queries shape: {nl_queries_embeddings.shape}")

All 12 DB columns:
Table: Player,Column: id,Description: unique ID for each player. used to identify and count how many players
Table: Player,Column: player_api_id,Description: API identifier for the player
Table: Player,Column: player_name,Description: full name of the player (first and last name)
Table: Player,Column: player_fifa_api_id,Description: unique FIFA-related identifier for the player
Table: Player,Column: birthday,Description: birthdate of the player format
Table: Player,Column: height,Description: height of the player in centimeters
Table: Player,Column: weight,Description: weight of the player in kilograms
Table: League,Column: id,Description: unique ID for each league. used to identify and count leagues
Table: League,Column: country_id,Description: foreign key that links the league to a country
Table: League,Column: name,Description: name of the league
Table: Country,Column: id,Description: unique ID for each country. links to league's country_id
Table: Country,Column: 

### Perform similarity search
[Top](#Document-overview)

**Facebook AI Similarity Search**<br>
This is an algorithm meant for finding semantic similarity between vectors. This means the model analyzes the actual meaning of the sentences rather than just comparing word similarity.

**K parameter**<br>
There is a parameter, K, that specifies how many search results will be returned. I talk about this later, but this is an opportunity for refinement, because ideally this should be dynamically generated based on the NLQ so that the model isn't feed too little/too much context.

In [3]:
# perform similarity search
def search_similarity(num_of_search_results=4):
    size_of_vectors = db_schema_embeddings.shape[1]
    index = faiss.IndexFlatL2(size_of_vectors)
    index.add(np.array(db_schema_embeddings))

    D, I = index.search(np.array(nl_queries_embeddings), k=num_of_search_results)

    all_search_results = []

    for i in I:
        matched_columns = []
        for j in i:
            matched_columns.append(db_columns[j])
        all_search_results.append(matched_columns)

    return all_search_results

print(f"Natural language query: {nl_queries[0]}")
print(f"\nSearch results:")
for column in search_similarity(num_of_search_results=2)[0]:
    print(column)


Natural language query: Who is the tallest player?

Search results:
Table: Player,Column: height,Description: height of the player in centimeters
Table: Player,Column: player_name,Description: full name of the player (first and last name)


### Convert search results to input prompt
[Top](#Document-overview)

**Parsing and reformatting search results**<br>
The search algorithm gave the information that I needed - the code below is a lot of re-formatting. I need the input to the model to be formatted as a table creation statement along with the NLQ, so there was a lot of parsing and reformatting that I had to do.

In [4]:
# utility functions
def create_db_connection():
    conn = sqlite3.connect("../eda/code/cyoung_eda.db")
    return conn, conn.cursor()

def create_input_prompt(nl_query, relevant_context):
    return f"tables:\n{relevant_context}\nquery for: {nl_query}"

def execute_query(query, print_padding=None):
    try:
        conn, cursor = create_db_connection()
        cursor.execute(query)

        if (print_padding):
            print(f"{'Query results:':<{print_padding}}{cursor.fetchall()}")
        return cursor.fetchall()
    except sqlite3.Error:
        print("Error executing sql")
    finally:
        conn.close()

In [5]:
# map search results to tables/columns
def create_table_statement(relevant_columns):
    tables_create_statements = {}
    table_and_cols_search_results = {}
    tables_in_search_results = list(set([result.split(",")[0].split(":")[1].strip() for result in relevant_columns]))
    for table in tables_in_search_results:
        table_columns = []
        for result in relevant_columns:
            if table == result.split(",")[0].split(":")[1].strip():
                table_columns.append(result.split(",")[1].split(":")[1].strip())
        if table in table_and_cols_search_results:
            table_and_cols_search_results[table].extend(table_columns)
        else:
            table_and_cols_search_results[table] = table_columns
        
    # format schema portion of T2SQL model input
    for table_name, columns in table_and_cols_search_results.items():
        conn, cursor = create_db_connection()
        cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name = '{table_name}'")
        raw_create_statement = cursor.fetchone()[0].encode().decode("unicode_escape").replace("`","")
        conn.close()
        filtered_create_lines = []
        create_lines = raw_create_statement.split("\n")
        for line in create_lines:
            line_segments = [segment.strip() for segment in line.split("\t")]
            if ("CREATE" in line_segments[0] or ")" == line_segments[0]):
                pass
            elif "FOREIGN" in line_segments[1]:
                create_col_name = re.findall(r"(?<=FOREIGN KEY\()[\w_]+(?=\))", line_segments[1])[0]
                if any(search_col_name == create_col_name for search_col_name in columns):
                    filtered_create_lines.append(create_col_name)
            else:
                create_col_name = line_segments[1]
                create_col_details = line_segments[2]
                if any(search_col_name == create_col_name for search_col_name in columns):
                    filtered_create_lines.append(f"{create_col_name} {create_col_details.replace(",", "")}")

        tables_create_statements[table_name] = f"CREATE TABLE {table_name} ({",".join(filtered_create_lines)})"
    return "\n".join(list(tables_create_statements.values()))

creation_statements = []
for search_result in search_similarity():
    creation_statements.append(create_table_statement(search_result))

input_prompts = []
for i in range(len(nl_queries)):
    input_prompts.append(
        create_input_prompt(
            nl_query=nl_queries[i],
            relevant_context=creation_statements[i]))
    
print(f"Example input prompt\n{input_prompts[2]}")

Example input prompt
tables:
CREATE TABLE Player (player_name TEXT,player_fifa_api_id INTEGER UNIQUE,height INTEGER,weight INTEGER)
query for: What is the name of the tallest player?


# Section 2 transfer learning
[Top](#Document-overview)

**Training/test split:**
I used an Excel file to generate my training data. I did this because the formulas made it easy to build templates and drag to duplicate records based on the template. The input to the training data is the like the example you see above in the previous section, including the relevant tables/columns and the NLQ. The output to the training data is the SQL query. I did an 80/20 train/test split.
<br><br>
**Preprocessing:**
I tokenize the data, and adjust parameters so that the data follows a unified format. For example, all inputs must be same length and all outputs must be same length. To achieve this, I pad when the value is too short, and also provide max length specifications


### Test train split and data preprocessing
[Top](#Document-overview)

In [6]:
# example https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)
model = AutoModelForSeq2SeqLM.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)

MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 128

input_output_examples = pd.read_excel("../data/data_sheets.xlsx", sheet_name="t2sql_data")


input_output_examples["search_results_formatted"] = input_output_examples["search_results"].fillna("").apply(lambda s: create_table_statement(s.split(";")))

input_output_examples["input"] = input_output_examples.apply(
    lambda row: create_input_prompt(row["natural_language_query"], row["search_results_formatted"]),
    axis=1)


train_df, test_df = train_test_split(input_output_examples, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df[["input", "output"]])
test_dataset = Dataset.from_pandas(test_df[["input", "output"]])

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

tokenizer = T5Tokenizer.from_pretrained("t5-small")


def preprocess_data(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length")
  
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["output"],
            padding="max_length",
            truncation=True,
            max_length=MAX_TARGET_LENGTH
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_data, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/48 [00:00<?, ? examples/s]

c:\Users\chris\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

### Hyperparameter tuning and training
[Top](#Document-overview)<br>
**Hyperparameter tuning**
I chose to experiment with 3 parameters
- Learning rate - how fast will learn. i.e. how big or small the adjustments will be
- Per device train batch size - how many examples will the model learn from at a time
- Weight decay - to avoid overfitting
<br><br>

**Training model**
- I evaluate the training data against the test data so that it can make adjustments

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

batch_size = 8
model_name = "t5-base-medium-title-generation"

args = Seq2SeqTrainingArguments(
    model_name,  # where the model will be saved to
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)
# https://huggingface.co/docs/transformers/en/hpo_train
# hyperparameters
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
    }

def model_init():
    return T5ForConditionalGeneration.from_pretrained("t5-small")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

best_trials = trainer.hyperparameter_search(
    direction=["minimize", "maximize"],
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=5,
)

trainer.train()

c:\Users\chris\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\chris\AppData\Local\Temp\ipykernel_7528\2284337687.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=6, training_loss=11.05084228515625, metrics={'train_runtime': 83.8352, 'train_samples_per_second': 0.573, 'train_steps_per_second': 0.072, 'total_flos': 6496406470656.0, 'train_loss': 11.05084228515625, 'epoch': 1.0})

# Section 3 putting it all together
[Top](#Document-overview)

**Successes**<br>
1. Dynamic context generation<br>
You can see here that I am not manually providing the relevant tables/columns to the model as context - the RAG implementation dynamically generates this.
2. Here I am using the fine-tuned model.

**Opportunities for improvement**
1. RAG refinements
You will notice that many of these queries are incorrect. This is because the # of query results being returned from the similarity search algorithm in [section 2](#perform-similarity-search) is not dynamically set. Some NLQs have 4 relevant columns, others only have 2, and if I give them 4 the model will be confused because of the noise like what you see here. This means I need to explore ways to dynamically calculate how many relevant columns should be returned for an NLQ.
2. Add more training data
While I have assembled the full pipeline to train data, you can see that it did not make a major impact. This is only because I have ~60 rows of training data. Now I need to generate more so that it can be impactful

In [8]:
def generate_sql(input_prompt, use_fine_tuned_model=False):
    # Load the tokenizer and model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = T5Tokenizer.from_pretrained('t5-small')
    # use fine-tuned model
    if use_fine_tuned_model:
        saved_model_path = "t5-base-medium-title-generation/checkpoint-6/"
        model = T5ForConditionalGeneration.from_pretrained(saved_model_path).to(device)
    else:
        model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql').to(device)

    """Generate SQL query from natural language input."""
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

padding = 30
for i in range(len(input_prompts)):
    sql_query = generate_sql(input_prompts[i])
    print(f"{'\nNatural language query:':<{padding}}{nl_queries[i]}")
    print(f"{'Generated SQL query:':<{padding}}{sql_query}")
    execute_query(sql_query, padding)



Natural language query:      Who is the tallest player?
Generated SQL query:          SELECT player_name FROM Player WHERE weight = (SELECT player_fifa_api_id FROM Player)
Query results:                []

Natural language query:      Who is the heaviest player?
Generated SQL query:          SELECT player_name FROM Player WHERE weight = (SELECT player_fifa_api_id FROM Player)
Query results:                []

Natural language query:      What is the name of the tallest player?
Generated SQL query:          SELECT player_name FROM Player WHERE weight = (SELECT player_fifa_api_id FROM Player)
Query results:                []

Natural language query:      How many players are there?
Generated SQL query:          SELECT COUNT(*) FROM Player
Query results:                [(11060,)]

Natural language query:      How many countries are there
Generated SQL query:          SELECT COUNT(*) FROM Country WHERE country_id IN (SELECT country_id FROM Player)
Error executing sql

Natural language que

# Section 4 resources
[Top](#Document-overview)

- [Fine tuning T5 model](https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887)
- [Hyperparameters with hugging face](https://huggingface.co/docs/transformers/en/hpo_train)
- [Text-to-sql model](https://huggingface.co/cssupport/t5-small-awesome-text-to-sql)
- [Using Facebook AI Similarity Search (FAISS)](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)